In [5]:
import pydrake.all
import numpy as np
import sympy as sym

In [2]:
#define constants
t, g= sym.symbols('t g')
mc, mp = sym.symbols('m_c m_p')
L, fx = sym.symbols('L f_x ')

# define variables 
theta,z = sym.symbols(r'\theta, z')
z_d = sym.symbols('z_d')
theta_d = sym.symbols(r'\theta_d')

In [5]:
z_dd = 1/(mc + mp*sym.sin(theta)**2)*(fx + mp*sym.sin(theta)*(L*(theta_d)**2+g*sym.cos(theta)))
theta_dd = 1/(L*(mc + mp*sym.sin(theta)**2))*(-fx*sym.cos(theta)-mp*L*(theta_d)**2*sym.cos(theta)*sym.sin(theta)-(mc+mp)*g*sym.sin(theta))

x_vec = sym.Matrix([z,theta,z_d,theta_d])
vector_field = sym.Matrix([z_d, theta_d,z_dd, theta_dd])

In [6]:
vector_field

Matrix([
[                                                                                                                      z_d],
[                                                                                                                 \theta_d],
[                                       (f_x + m_p*(L*\theta_d**2 + g*cos(\theta))*sin(\theta))/(m_c + m_p*sin(\theta)**2)],
[(-L*\theta_d**2*m_p*sin(\theta)*cos(\theta) - f_x*cos(\theta) - g*(m_c + m_p)*sin(\theta))/(L*(m_c + m_p*sin(\theta)**2))]])

In [7]:
# Linearization
# calculate the Jacobian around equilibrim point
pfpx = vector_field.jacobian([z, theta, z_d, theta_d])
pfpu = vector_field.jacobian([fx])

In [9]:
pfpx


Matrix([
[0,                                                                                                                                                                                                                                                                                                                0, 1,                                                                  0],
[0,                                                                                                                                                                                                                                                                                                                0, 0,                                                                  1],
[0,                                                                                      -2*m_p*(f_x + m_p*(L*\theta_d**2 + g*cos(\theta))*sin(\theta))*sin(\theta)*cos(\theta)/(m_c + m_p*sin(\theta)**2)**2 + (-g*m_p*sin(\theta)

In [10]:
pfpu

Matrix([
[                                          0],
[                                          0],
[               1/(m_c + m_p*sin(\theta)**2)],
[-cos(\theta)/(L*(m_c + m_p*sin(\theta)**2))]])

In [17]:
# Convert a SymPy expression into a function
function_pfpx = sym.lambdify((g,fx,mc,mp,L,theta,theta_d,z,z_d), pfpx)
funciton_pfpu = sym.lambdify((g,fx,mc,mp,L,theta,theta_d,z,z_d), pfpu)
Ahat = function_pfpx(9.81,0,10,1,0.5,np.pi,0,0,0)
Bhat = funciton_pfpu(9.81,0,10,1,0.5,np.pi,0,0,0)

In [18]:
Ahat 

array([[ 0.   ,  0.   ,  1.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  1.   ],
       [ 0.   ,  0.981,  0.   ,  0.   ],
       [ 0.   , 21.582,  0.   , -0.   ]])

In [19]:
Bhat

array([[0. ],
       [0. ],
       [0.1],
       [0.2]])

In [24]:
import scipy.signal as sig
cartpole_gain = sig.place_poles(Ahat, Bhat,  [-1+2*1j, -1-2*1j, -3, -2]).gain_matrix

In [25]:
cartpole_gain

array([[-15.29051988, 220.55525994, -18.85830785,  44.42915392]])

In [40]:
from pydrake.all import LinearQuadraticRegulator, LinearSystem
Q = np.diag((10., 10., 1., 1.))
R = [1]
Chat = np.mat('0,0,0,0')
Dhat = [0]
cartpole_syst = LinearSystem(Ahat, Bhat, Chat, Dhat)
lqr = LinearQuadraticRegulator(cartpole_syst, Q, R)

In [1]:
server_args = []
import math
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)
from pydrake.all import (LeafSystem,BasicVector,DiagramBuilder, AddMultibodyPlantSceneGraph, Parser, LinearQuadraticRegulator,
                         Simulator, RigidTransform, CoulombFriction, FindResourceOrThrow, DrakeVisualizer, ConnectContactResultsToDrakeVisualizer,
                         RollPitchYaw, JointIndex, namedview, ConnectMeshcatVisualizer,
                         Value, List, ZeroOrderHold, SpatialAcceleration, RotationMatrix, AbstractValue, ConstantVectorSource)

In [3]:
def UprightState():
        state = [0, np.pi, 0, 0]
        return state

def BalancingLQR(plant):
    # Design an LQR controller for stabilizing the CartPole around the upright.
    # Returns a (static) AffineSystem that implements the controller (in
    # the original CartPole coordinates).

    context = plant.CreateDefaultContext()
    plant.get_actuation_input_port().FixValue(context, [0])
    context.get_mutable_continuous_state_vector().SetFromVector(UprightState())
    Q = np.diag((10., 10., 1., 1.))
    R = [1]     

    # MultibodyPlant has many (optional) input ports, so we must pass the
    # input_port_index to LQR.
    return LinearQuadraticRegulator(
        plant,
        context,
        Q,
        R,
        input_port_index=plant.get_actuation_input_port().get_index(0))

class myController(LeafSystem):
    def __init__(self, K):
        LeafSystem.__init__(self)                  
        self.DeclareVectorInputPort("u", BasicVector(4))        
        self.DeclareVectorOutputPort("y", BasicVector(1), self.CalcOutputY) 
        self.K = K                                
    def CalcOutputY(self, context, output):
        statex = self.get_input_port(0).Eval(context)         
        y = -np.dot(self.K, (statex-np.array([0, np.pi, 0, 0])))    
      #  print(statex, y, statex-np.array([0, np.pi, 0, 0]))                         
        output.SetFromVector([y])

In [8]:
from pydrake.all import (BasicVector, LeafSystem, DiagramBuilder, Simulator, LogOutput, ConstantVectorSource, 
                         LinearSystem)

K = np.array([-15.29051988, 220.55525994, -18.85830785,  44.42915392])
builder = DiagramBuilder()
controller = builder.AddSystem(myController(K))
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
# Select a robot model
urdf_path = "./urdfExample_cart_pole.urdf"    
cart_pole = Parser(plant, scene_graph).AddModelFromFile(urdf_path)    
plant.Finalize()

builder.Connect(plant.get_state_output_port(), controller.get_input_port(0))
builder.Connect(controller.get_output_port(), plant.get_actuation_input_port())

logger_pendulum_state = LogOutput(plant.get_state_output_port(), builder)

meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, open_browser=True)
diagram = builder.Build()

simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1)
context = simulator.get_mutable_context()
context.SetContinuousState(UprightState() + np.array([0,0.2,0,0]))
simulator.Initialize()
sim_time = 5
meshcat.start_recording() 
simulator.AdvanceTo(sim_time)
meshcat.stop_recording()
meshcat.publish_recording()


/mnt/c/Users/harlab/code/MEE424/project2_cartpole/env/lib/python3.8/site-packages/pydrake/common/cpp_template.py:392: DrakeDeprecationWarning: (Deprecated.)

Deprecated:
    Use LogVectorOutput instead. This will be removed from Drake on or
    after 2021-12-01.
  def f(*args, **kwargs): return orig(*args, **kwargs)


Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.
